This notebook goes over the code needed to reproduce the results from the MSE and time figures in the first two simulations in the paper: the Normal Normal Log-Normal simulation and the Multivariate T, Multivariate Gamma, Inverse-Gamma simulation. 

The provided code produces results given in Figures 2 and 3, in addition to Supplementary Tables S.1 and S.2.

The simulations were run in Julia version 1.4.0. The `randn!()` function changed in v1.5.0, and will produce different random numbers for the simulation. 

In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 8


### Normal-Normal Log-Normal Simulation

The following code runs the simulation and saves the results.

In [ ]:
using WiSER, DelimitedFiles, LinearAlgebra, Random, KNITRO
using Random, Roots, SpecialFunctions

function MSE(m::WSVarLmmModel, β::Vector, τ::Vector, Σγ::Matrix)
    mseβ = sum(abs2, m.β - β) / m.p
    mseτ = sum(abs2, m.τ[2:end] - τ[2:end]) / (m.l - 1)
    msevechLγ = sum(abs2, m.Σγ - Σγ) / abs2(m.q)
    return mseβ, mseτ, msevechLγ
end

function coverage!(m::WSVarLmmModel, trueparams::Vector, 
    intervals::Matrix, curcoverage::Vector)

    copyto!(intervals, confint(m))
    lbs = @views intervals[:, 1]
    ubs = @views intervals[:, 2]
    map!((val, lb, ub) -> val >= lb && 
        val <= ub, curcoverage, trueparams, lbs, ubs)
    return curcoverage
end

# dimensions
@show Threads.nthreads()
p  = 5    # number of fixed effects, including intercept
q  = 3    # number of random effects, including intercept
l  = 5    # number of WS variance covariates, including intercept
# true parameter values
βtrue = [ 0.1; 6.5; -3.5; 1.0; 5  ]
τtrue = [0.0; 0.5; -0.2; 0.5; 0.0]
Σγ    = [1.5 0.5 0.3;
         0.5 1.0 0.2;
         0.3 0.2 0.5];
δγω   = [0.2; 0.1; 0.05]
σω    = [0.5]
Σγω   = [Σγ δγω; δγω' σω]
Lγω   = cholesky(Symmetric(Σγω), check = false).L
Lγ    = Lγω[1:q, 1:q]
lγω   = Lγω[q + 1, 1:q]
lω    = Lγω[q + 1, q + 1]
# generate data
γω = Vector{Float64}(undef, q + 1)
z  = similar(γω) # hold vector of iid std normal

intervals = zeros(p + l, 2) #hold intervals
curcoverage = zeros(p + l) #hold current coverage resutls
trueparams = [βtrue; τtrue] #hold true parameters

#simulation parameters
samplesizes = collect(1000:1000:6000)
ns = [10; 25; 50; 100; 1000]
nsims = 200

#storage for results
βMseResults = ones(nsims * length(ns) * length(samplesizes))
τMseResults = ones(nsims * length(ns) * length(samplesizes))
ΣγMseResults = ones(nsims * length(ns) *  length(samplesizes))
βτcoverage = Matrix{Float64}(undef, p + l, nsims * length(ns) * length(samplesizes))
fittimes = zeros(nsims * length(ns) * length(samplesizes))
solver = KNITRO.KnitroSolver(outlev=0)

st = time()
currentind = 1
for t in 1:length(samplesizes)
    m = samplesizes[t]
    obsvec = Vector{WSVarLmmObs{Float64}}(undef, m)
    for k in 1:length(ns)
        ni = ns[k] # number of observations per individual
        y = Vector{Float64}(undef, ni)
        for j in 1:nsims
            println("rep $j obs per person $ni samplesize $m")
            Random.seed!(j + 100000k + 1000t)
            for i in 1:m
                # first column intercept, remaining entries iid std normal
                X = Matrix{Float64}(undef, ni, p)
                X[:, 1] .= 1
                @views randn!(X[:, 2:p])
                # first column intercept, remaining entries iid std normal
                Z = Matrix{Float64}(undef, ni, q)
                Z[:, 1] .= 1
                @views randn!(Z[:, 2:q])
                # first column intercept, remaining entries iid std normal
                W = Matrix{Float64}(undef, ni, l)
                W[:, 1] .= 1
                @views randn!(W[:, 2:l])
                # generate random effects: γω = Lγω * z
                mul!(γω, Lγω, randn!(z))
                # generate y
                μy = X * βtrue + Z * γω[1:q]
                @views ysd = exp.(0.5 .* (W * τtrue .+ dot(γω[1:q], lγω) .+ γω[end]))
                y = ysd .* randn(ni) .+ μy
                # form a VarLmmObs instance
                obsvec[i] = WSVarLmmObs(y, X, Z, W)
            end
            # form VarLmmModel
            vlmm = WSVarLmmModel(obsvec);
            fittime = NaN
            try 
                fittime = @elapsed WiSER.fit!(vlmm, solver, parallel=false)
                confint(vlmm)
            catch
                println("rep $j ni obs = $ni , samplesize = $m had an error")
                try 
                    fittime = @elapsed WiSER.fit!(vlmm, Ipopt.IpoptSolver(print_level=1, 
                        watchdog_shortened_iter_trigger=3, max_iter=100), parallel=false)
                        confint(vlmm)
                catch
                    println("rep $j ni obs = $ni , samplesize = $m had a second error")
                    try
                        fittime = @elapsed WiSER.fit!(vlmm, init = init_mom!(vlmm), parallel=false)
                        confint(vlmm)
                    catch
                        println("rep $j ni obs = $ni , samplesize = $m had a third error")
                        βMseResults[currentind] = NaN
                        τMseResults[currentind] = NaN
                        ΣγMseResults[currentind] = NaN
                        βτcoverage[:, currentind] .= NaN
                        fittimes[currentind] = NaN
                        currentind += 1
                        continue
                    end
                end
            end
            coverage!(vlmm, trueparams, intervals, curcoverage)
            mseβ, mseτ, mseΣγ = MSE(vlmm, βtrue, τtrue, Σγ)

            #index = Int(nsims * length(ns) * (t - 1) + nsims * (k - 1) + j)
            global currentind
            @views copyto!(βτcoverage[:, currentind], curcoverage)
            βMseResults[currentind] = mseβ
            τMseResults[currentind] = mseτ
            ΣγMseResults[currentind] = mseΣγ
            fittimes[currentind] = fittime
            currentind += 1
        end
    end
end 
en = time()

@show en - st #seconds 

ftail = "normal_normal_lognormal_$(nsims)reps_sim.csv"
writedlm("mse_beta_" * ftail, βMseResults, ',')
writedlm("mse_tau_" * ftail, τMseResults, ',')
writedlm("mse_Sigma_" * ftail, ΣγMseResults, ',')
writedlm("beta_tau_coverage_5betas_" * ftail, βτcoverage, ',')
writedlm("fittimes" * ftail, fittimes, ',')

The following code will create the Time figure (Figure 2).

In [ ]:
using Random, Gadfly, DataFrames, DelimitedFiles, Statistics, RCall, Printf
import StatsBase: sem
ENV["COLUMNS"]=1000

p = 5; l = 5
sample_sizes = repeat(string.(samplesizes), inner = nsims * length(ns))
obs_sizes = repeat(string.(ns), inner = nsims, outer = length(samplesizes))
msedf = DataFrame(βmse = βMseresult, τmse = τMseresult, Σγmse = ΣMseresult, 
    samplesize = sample_sizes, obssize = obs_sizes, fittimes = fittimes)
timedf = combine(groupby(msedf, [:samplesize, :obssize]), :fittimes => mean => :fittime,
    :fittimes => x -> (std(x)/sqrt(length(x))))
rename!(timedf, Dict(:fittimes_function => "se"))
timedf[!, :nobs] = Meta.parse.(timedf[!, :samplesize]) .* Meta.parse.(timedf[!, :obssize])
timedf[!, :fitmin] = timedf[!, :fittime] - timedf[!, :se]
timedf[!, :fitmax] = timedf[!, :fittime] + timedf[!, :se]
timedf[!, :perobsratio] = timedf[!, :fittime] ./ timedf[!, :nobs]
timedf

#### Time figure

In [ ]:
using RCall
@rput timedf

R"""
library(ggplot2)
timedf$obssize <- factor(timedf$obssize, levels = c('10', '25', '50', '100', '1000'))
timedf$samplesize <- factor(timedf$samplesize, levels = c('1000', '2000', '3000', '4000', '5000', '6000'))

fittime_1 = ggplot(timedf, aes(x=samplesize, y=fittime, group=obssize, color=obssize)) + 
  geom_line() +
  geom_point()+
  geom_errorbar(aes(ymin=fitmin, ymax=fitmax), width=0.5, alpha = 0.8, position=position_dodge(0.005)) + 
  theme(legend.position=c(0.15,0.85), legend.key = element_blank(), axis.text.x = element_text(angle=0, size=13),
        axis.text.y = element_text(angle=0, size=13), axis.title.x = element_text(size = 17), 
        axis.title.y = element_text(size = 17), legend.title = element_text(size = 14),
        #axis.ticks = element_blank(),
        panel.grid.major = element_blank(), legend.text=element_text(size=13),
        panel.border = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank(), 
        axis.line = element_line(color = 'black',size=0.3), plot.title = element_text(hjust = 0.5)) + 
   scale_color_manual(values = c("#c85f55",
"#a964bf",
"#8db352",
"#fa7300",
"#05aec0")) +
  labs(x = "Number of Individuals", y = "Fit time (seconds)", color = "Obs per Individual")


"""

In [ ]:
R"""
ggsave('singlethread_fittimes.png', fittime_1, width = 8, height = 8, units = 'in', dpi = 450)
"""

In [ ]:
using RCall
@rput timedf

R"""
library(scales)
library(ggplot2)
timedf$obssize <- factor(timedf$obssize, levels = c('10', '25', '50', '100', '1000'))
timedf$samplesize <- factor(timedf$samplesize, levels = c('1000', '2000', '3000', '4000', '5000', '6000'))

fancy_scientific <- function(l) {
     # turn in to character string in scientific notation
     l <- format(l, scientific = TRUE)
     # quote the part before the exponent to keep all the digits
     l <- gsub("^(.*)e", "'\\1'e", l)
     # turn the 'e+' into plotmath format
     l <- gsub("e", "%*%10^", l)
     # return this as an expression
     parse(text=l)
}

fittimeperobs = ggplot(timedf, aes(x=nobs, y=perobsratio)) + 
  geom_line() +
  geom_point()+
#  geom_errorbar(aes(ymin=fitmin, ymax=fitmax), width=0.5, alpha = 0.8, position=position_dodge(0.005)) + 
  theme(legend.position=c(0.15,0.8), legend.key = element_blank(), axis.text.x = element_text(angle=0, size=13),
        axis.text.y = element_text(angle=0, size=13), axis.title.x = element_text(size = 17), 
        axis.title.y = element_text(size = 17), legend.title = element_text(size = 14),
        #axis.ticks = element_blank(),
        panel.grid.major = element_blank(), legend.text=element_text(size=11),
        panel.border = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank(), 
        axis.line = element_line(color = 'black',size=0.3), plot.title = element_text(hjust = 0.5)) + 
  labs(x = "Total Number of Observations", y = "Fit time per observation (seconds)", color = "Obs per Individual") +
#scale_x_log10(breaks = 10^seq(0, 7, 1), labels=trans_format("log10", math_format(10^.x)))# + #, limit=c(10^0, 10^7))
scale_x_continuous(breaks = seq(0, 6000000, 1000000), labels= fancy_scientific) +
scale_y_continuous(breaks = c(10^-5, 2 * 10^-5, 3 * 10^-5, 4 * 10^-5, 5 * 10^-5, 6 * 10^-5), labels= fancy_scientific)

"""

In [ ]:
R"""
ggsave('singlethread_fittimeperobs.png', fittimeperobs, width = 8, height = 8, units = 'in', dpi = 450)
"""

##### Supplementary Table S.1

The following code produces results seen in the Supplementary Table S.1.

In [ ]:
using Random, Gadfly, DataFrames, DelimitedFiles, Statistics
import StatsBase: sem
ENV["COLUMNS"]=800

βτcoverage = readdlm("beta_tau_coverage_5betas_normal_normal_lognormal_200reps_sim.csv", ',')
samplesizes = collect(1000:1000:6000)
ns = [10; 25; 50; 100; 1000]
nsims = 200

covdf = DataFrame(transpose(βτcoverage))
rename!(covdf, Symbol.([["β$i" for i in 1:p]; ["τ$i" for i in 1:l]]))
covdf[!, :samplesize] = sample_sizes
covdf[!, :obssize] = obs_sizes
first(covdf, 10)

row_stats = [[(mean(col), sem(col)) for col = eachcol(d[!, 1:end-2])] for d = groupby(covdf, [:samplesize; :obssize])]
df = DataFrame(row_stats)
ss_obs = unique("m: " .* sample_sizes .* "  ni: " .* obs_sizes)
rename!(df, Symbol.(ss_obs))
covdfdisplay = DataFrame([[names(df)]; collect.(eachrow(df))], 
    [:people_obs; Symbol.([["β$i" for i in 1:p]; ["τ$i" for i in 1:l]])])
deletecols!(covdfdisplay, p + 2)
covdfdisplay

### Multivariate T, Multivariate Gamma, Inverse-Gamma simulation. 

The following code runs the simulation and saves the results.

In [ ]:
using WiSER, DelimitedFiles, LinearAlgebra, Random, KNITRO
using Random, Roots, SpecialFunctions

function MSE(m::WSVarLmmModel, β::Vector, τ::Vector, Σγ::Matrix)
    mseβ = sum(abs2, m.β - β) / m.p
    mseτ = sum(abs2, m.τ[2:end] - τ[2:end]) / (m.l - 1)
    msevechLγ = sum(abs2, m.Σγ - Σγ) / abs2(m.q)
    return mseβ, mseτ, msevechLγ
end

function coverage!(m::WSVarLmmModel, trueparams::Vector, 
    intervals::Matrix, curcoverage::Vector)
    copyto!(intervals, confint(m))
    lbs = @views intervals[:, 1]
    ubs = @views intervals[:, 2]
    map!((val, lb, ub) -> val >= lb && 
        val <= ub, curcoverage, trueparams, lbs, ubs)
    return curcoverage
end

# dimensions
@show Threads.nthreads()
p  = 5    # number of fixed effects, including intercept
q  = 3    # number of random effects, including intercept
l  = 5    # number of WS variance covariates, including intercept
# true parameter values
βtrue = [ 0.1; 6.5; -3.5; 1.0; 5]
τtrue = [0.0; 0.5; -0.2; 0.5; 0.0]#[-1.5; 1.5; -0.5; 0.0; 0.0]
Σγ    = [1.5 0.5 0.3;
         0.5 1.0 0.2;
         0.3 0.2 0.5];
δγω   = [0.0; 0.0; 0.0] # for easier generation of (γ, ω)
σω    = 0.5
Σγω   = [Σγ δγω; δγω' σω]
Lγω   = cholesky(Symmetric(Σγω), check = false).L
Lγ    = Lγω[1:q, 1:q]
lγω   = Lγω[q + 1, 1:q]
lω    = Lγω[q + 1, q + 1]
# parameters for Gamma random deviate ω
# If X ∼ Gamma(α, θ), then E[-ln(X)] = - ψ(α) - ln(θ), Var[-ln(X)] = ψ'(α).
# ωi = log(inv(X)) = - log(X)
# We want Var[ωi] = ψ'(α) = σω and E[ωi] = - ψ(α) - ln(θ) = 0
ωα    = Roots.find_zero(x -> trigamma(x) - σω, 1)
ωα > 1 || error("ωα needs to be >1 for the existence of mean of inverse-gamma")
ωθ    = exp(-digamma(ωα))
# parameters for Gamma random deviate γ
# If X ∼ Gamma(α, θ), then E[X] = αθ, Var[X] = αθ^2.
# We want Var[X] = 1 and don't care about mean (will shift it to 0)
γα    = 4             # shape parameter 
γθ    = sqrt(inv(γα)) # scale parameter
# degree of freedom for t
ν     = 6
# generate data
γ     = Vector{Float64}(undef, q)
z     = similar(γ) # hold vector of iid gamma

intervals = zeros(p + l, 2) #hold intervals
curcoverage = zeros(p + l) #hold current coverage resutls
trueparams = [βtrue; τtrue] #hold true parameters

#simulation parameters
samplesizes = collect(1000:1000:6000)
ns = [10; 25; 50; 100; 1000]
nsims = 200

#storage for results
βMseResults = ones(nsims * length(ns) * length(samplesizes))
τMseResults = ones(nsims * length(ns) * length(samplesizes))
ΣγMseResults = ones(nsims * length(ns) *  length(samplesizes))
βτcoverage = Matrix{Float64}(undef, p + l, nsims * length(ns) * length(samplesizes))
fittimes = zeros(nsims * length(ns) * length(samplesizes))
solver = KNITRO.KnitroSolver(outlev=0)

st = time()
currentind = 1
for t in 1:length(samplesizes)
    m = samplesizes[t]
    obsvec = Vector{WSVarLmmObs{Float64}}(undef, m)
    for k in 1:length(ns)
        ni = ns[k] # number of observations per individual
        y = Vector{Float64}(undef, ni)
        for j in 1:nsims
            println("rep $j obs per person $ni samplesize $m")
            Random.seed!(j + 100000k + 1000t)
            for i in 1:m
                # first column intercept, remaining entries iid std normal
                X = Matrix{Float64}(undef, ni, p)
                X[:, 1] .= 1
                @views randn!(X[:, 2:p])
                # first column intercept, remaining entries iid std normal
                Z = Matrix{Float64}(undef, ni, q)
                Z[:, 1] .= 1
                @views randn!(Z[:, 2:q])
                # first column intercept, remaining entries iid std normal
                W = Matrix{Float64}(undef, ni, l)
                W[:, 1] .= 1
                @views randn!(W[:, 2:l])
                # generate ω ∼ log-inv-gamma(ωα, ωθ)
                ω = -log(rand(Gamma(ωα, ωθ)))
                # generate random effects: γ = Lγ * z
                # z is iid Gamma with variance 1 and shifted to have mean 0
                Distributions.rand!(Gamma(γα, γθ), z)
                z .-= γα * γθ # shift to have mean 0
                mul!(γ, Lγ, z)
                # generate y from t distribution (ν, μy, σ2ϵ)
                μy  = X * βtrue + Z * γ
                σ2ϵ = W * τtrue .+ dot(γ, lγω) .+ ω
                ysd = exp.(0.5 .* (σ2ϵ))
                # note: variance of T(ν) is ν / (ν - 2)
                y = μy + sqrt(((ν - 2) / ν)) .* ysd .* rand(TDist(ν), ni)
                # form a VarLmmObs instance
                obsvec[i] = WSVarLmmObs(y, X, Z, W)
            end
            # form VarLmmModel
            vlmm = WSVarLmmModel(obsvec);
            fittime = NaN
            try 
                fittime = @elapsed WiSER.fit!(vlmm, solver, parallel = false)
                confint(vlmm)
            catch
                println("rep $j ni obs = $ni , samplesize = $m had an error")
                try 
                    fittime = @elapsed WiSER.fit!(vlmm, Ipopt.IpoptSolver(print_level=1, 
                        watchdog_shortened_iter_trigger=3, max_iter=100), parallel = false)
                        confint(vlmm)
                catch
                    println("rep $j ni obs = $ni , samplesize = $m had a second error")
                    try
                        fittime = @elapsed WiSER.fit!(vlmm, init = init_mom!(vlmm), parallel = false)
                        confint(vlmm)
                    catch
                        println("rep $j ni obs = $ni , samplesize = $m had a third error")
                        βMseResults[currentind] = NaN
                        τMseResults[currentind] = NaN
                        ΣγMseResults[currentind] = NaN
                        βτcoverage[:, currentind] .= NaN
                        fittimes[currentind] = NaN
                        currentind += 1
                        continue
                    end
                end
            end
            coverage!(vlmm, trueparams, intervals, curcoverage)
            mseβ, mseτ, mseΣγ = MSE(vlmm, βtrue, τtrue, Σγ)

            #index = Int(nsims * length(ns) * (t - 1) + nsims * (k - 1) + j)
            global currentind
            @views copyto!(βτcoverage[:, currentind], curcoverage)
            βMseResults[currentind] = mseβ
            τMseResults[currentind] = mseτ
            ΣγMseResults[currentind] = mseΣγ
            fittimes[currentind] = fittime
            currentind += 1
        end
    end
end 
en = time()

@show en - st #seconds 

ftail = "mvt_gamma_invgamma_$(nsims)reps_sim.csv"
writedlm("mse_beta_" * ftail, βMseResults, ',')
writedlm("mse_tau_" * ftail, τMseResults, ',')
writedlm("mse_Sigma_" * ftail, ΣγMseResults, ',')
writedlm("beta_tau_coverage_5betas_" * ftail, βτcoverage, ',')
writedlm("fittimes" * ftail, fittimes, ',')

#### MSE Figure

The following reproduces Figure 3 (the MSE Figure) that uses results from both simulations.

In [ ]:
#import data and reorganize to create figure
using Random, Gadfly, DataFrames, DelimitedFiles, Statistics, RCall, Printf
import StatsBase: sem
ENV["COLUMNS"]=1000

βMseresultrobust = vec(readdlm("mse_beta_mvt_gamma_invgamma_200reps_sim.csv", ','))
τMseresultrobust = vec(readdlm("mse_tau_mvt_gamma_invgamma_200reps_sim.csv", ','))
ΣMseresultrobust = vec(readdlm("mse_Sigma_mvt_gamma_invgamma_200reps_sim.csv", ','))
βτcoveragerobust = readdlm("beta_tau_coverage_5betas_mvt_gamma_invgamma_200reps_sim.csv", ',')
fittimesrobust = vec(readdlm("fittimesmvt_gamma_invgamma_200reps_sim.csv", ','))


samplesizesrobust = collect(1000:1000:6000)
nsrobust = [10; 25; 50; 100; 1000]
nsimsrobust = 200

βMseresultnorm = vec(readdlm("mse_beta_normal_normal_lognormal_200reps_sim.csv", ','))
τMseresultnorm = vec(readdlm("mse_tau_normal_normal_lognormal_200reps_sim.csv", ','))
ΣMseresultnorm = vec(readdlm("mse_Sigma_normal_normal_lognormal_200reps_sim.csv", ','))
βτcoveragenorm = readdlm("beta_tau_coverage_5betas_normal_normal_lognormal_200reps_sim.csv", ',')
fittimesnorm = vec(readdlm("fittimesnormal_normal_lognormal_200reps_sim.csv", ','))


samplesizesnorm = collect(1000:1000:6000)
nsnorm = [10; 25; 50; 100; 1000]
nsimsnorm = 200

p = 5; l = 5
sample_sizesrobust = repeat(string.(samplesizesrobust), inner = nsimsrobust * length(nsrobust))
obs_sizesrobust = repeat(string.(nsrobust), inner = nsimsrobust, outer = length(samplesizesrobust))
msedfrobust = DataFrame(βmse = βMseresultrobust, τmse = τMseresultrobust, Σγmse = ΣMseresultrobust, 
    samplesize = sample_sizesrobust, obssize = obs_sizesrobust)

p = 5; l = 5
sample_sizesnorm = repeat(string.(samplesizesnorm), inner = nsimsnorm * length(nsnorm))
obs_sizesnorm = repeat(string.(nsnorm), inner = nsimsnorm, outer = length(samplesizesnorm))
msedfnorm = DataFrame(βmse = βMseresultnorm, τmse = τMseresultnorm, Σγmse = ΣMseresultnorm, 
    samplesize = sample_sizesnorm, obssize = obs_sizesnorm)

#rename to make parsing easier for R.

msedfrobustR = deepcopy(msedfrobust)
rename!(msedfrobustR, ["betamse"
    "taumse"
    "Sigmamse"
    "samplesize"
 "obssize"]);

msedfnormR = deepcopy(msedfnorm)
rename!(msedfnormR, ["betamse"
    "taumse"
    "Sigmamse"
    "samplesize"
 "obssize"]);

mses = [msedfrobustR[!, :betamse]; msedfrobustR[!, :taumse]; msedfrobustR[!, :Sigmamse];  
    msedfnormR[!, :betamse]; msedfnormR[!, :taumse]; msedfnormR[!, :Sigmamse]]
obssize = [repeat(msedfrobustR[!, :obssize], 3); repeat(msedfnormR[!, :obssize], 3)]
samplesize = [repeat(msedfrobustR[!, :samplesize], 3); repeat(msedfnormR[!, :samplesize], 3)]

parameters = [repeat(string.([:beta, :tau, :Sigma]), inner = nsimsrobust * length(nsrobust) * length(samplesizesrobust)); 
    repeat(string.([:beta, :tau, :Sigma]), inner = nsimsnorm * length(nsnorm) * length(samplesizesnorm))]
robust = [repeat(["MvT Gamma Inverse Gamma"], 3 * nsimsrobust * length(nsrobust) * length(samplesizesrobust));
repeat(["Normal Normal Log-Normal"], 3 * nsimsnorm * length(nsnorm) * length(samplesizesnorm))];

msedfR = DataFrame(mse = mses, obssize = obssize, samplesize = samplesize,
    parameters = parameters, robust = robust)

Following creates the figure:

In [ ]:
@rput msedfR

R"""
library(scales)
library(ggplot2)
library(facetscales)
library(data.table)

#one large beta outlier (483) index 1141
msedfR = msedfR[-1141, ]

msedfR <- data.table(msedfR)

msedfR[parameters == "beta",y_min := 10^-8]
msedfR[parameters == "beta",y_max := 10^-1]
msedfR[parameters == "tau",y_min := 10^-8]
msedfR[parameters == "tau",y_max := 10^2]
msedfR[parameters == "Sigma",y_min := 10^-5]
msedfR[parameters == "Sigma",y_max := 10^0]


msedfR$obssize <- factor(msedfR$obssize, levels = c('10', '25', '50', '100', '1000'))
msedfR$samplesize <- factor(msedfR$samplesize, levels = c('1000', '2000', '3000', '4000', '5000', '6000'))
msedfR$parameters <- factor(msedfR$parameters, levels = c('beta', 'tau', 'Sigma'),
    labels = c(beta = expression(hat(bold(beta))), tau = expression(hat(bold(tau))), 
            Sigma = expression(hat(bold(Sigma))[bold(gamma)])))
msedfR$robust <- factor(msedfR$robust, levels = c('Normal Normal Log-Normal', 'MvT Gamma Inverse Gamma'),
    labels = c(expression(paste("Normal Normal Log-Normal")), expression(paste("MvT Gamma Inverse-Gamma"))))


mseplot <- ggplot(msedfR[msedfR$mse < 10^3, ], aes(x=samplesize, y=mse, fill=obssize)) + 
  geom_boxplot(outlier.size=0.5) +
    facet_grid(parameters ~ robust, labeller = label_parsed, scales = "free_y") +
  theme(legend.position="right", legend.key = element_blank(), axis.text.x = element_text(angle=0, size=11),
        axis.text.y = element_text(angle=0, size=12), axis.title.x = element_text(size = 15), 
        axis.title.y = element_text(size = 15), legend.title = element_text(size = 12),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        legend.text=element_text(size=10), panel.background = element_rect(fill = NA, color = "black"),
        #panel.background = element_blank(), #panel.border = element_blank(),
        axis.line = element_line(color = 'black',size=0.3), plot.title = element_text(hjust = 0.5),
        strip.background = element_rect(colour="black", fill="white"), strip.text.y = element_text(size=15, face="bold"),
        strip.text.x = element_text(size=15)) + 
  scale_fill_manual(values = c("#c85f55",
"#a964bf",
"#8db352",
"#fa7300",
"#05aec0")) +
#scale_y_log10(breaks = scales::trans_breaks("log10", function(x) 10^x)) +
#scale_y_log10(breaks = 10^(-8:8), limit=c(10^-8, 10^2)) + 
scale_y_log10(breaks = 10^seq(-10, 10, 2), labels=trans_format("log10", math_format(10^.x))) + #, limit=c(10^-8, 10^2)) +
  labs(x = "Number of Individuals", y = "MSE of Parameter Estimates", fill = "Obs per Individual") +
geom_blank(aes(y = y_max)) + 
geom_blank(aes(y = y_min)) 

"""

In [ ]:
R"""
ggsave('msefacetplot.png', mseplot, width = 12.5, height = 15, units = 'in', dpi = 450)
"""

### Supplementary Tables S.1 and S.2

The following produces coverage tables from Supplementary Materials for the two simulations.

##### Supplementary Table S.2

The following code produces results seen in the Supplementary Table S.2.

In [ ]:
using Random, Gadfly, DataFrames, DelimitedFiles, Statistics
import StatsBase: sem
ENV["COLUMNS"]=800

βτcoverage = readdlm("beta_tau_coverage_5betas_mvt_gamma_invgamma_200reps_sim.csv", ',')
samplesizes = collect(1000:1000:6000)
ns = [10; 25; 50; 100; 1000]
nsims = 200

covdf = DataFrame(transpose(βτcoverage))
rename!(covdf, Symbol.([["β$i" for i in 1:p]; ["τ$i" for i in 1:l]]))
covdf[!, :samplesize] = sample_sizes
covdf[!, :obssize] = obs_sizes
first(covdf, 10)

row_stats = [[(mean(col), sem(col)) for col = eachcol(d[!, 1:end-2])] for d = groupby(covdf, [:samplesize; :obssize])]
df = DataFrame(row_stats)
ss_obs = unique("m: " .* sample_sizes .* "  ni: " .* obs_sizes)
rename!(df, Symbol.(ss_obs))
covdfdisplay = DataFrame([[names(df)]; collect.(eachrow(df))], 
    [:people_obs; Symbol.([["β$i" for i in 1:p]; ["τ$i" for i in 1:l]])])
deletecols!(covdfdisplay, p + 2)
covdfdisplay